## Loads a continuous trajectory and discretizes it

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.trajectoryTools as trajectoryTools

In [ ]:
# Parameters for loading continuous trajectories from files (should match values used to compute trajectories)
nfiles = 500
dt = 0.00001 # from original simulation
stride = 25  # stride of sampling (every 25th dt)
totalTimeSteps = 2000000
boxsize = 4
boundaryType = 'periodic'

# Calculated parameters
dtEffective = dt*stride # needed to obtain rate dictionary
effectivetimeSteps = int(totalTimeSteps/25)
fnamebase = '../data/dimer/simDimer_t' + "{:.2E}".format(totalTimeSteps) + "_s{:d}".format(stride) + "_"

In [ ]:
# Load trajectories
trajs = []
for i in range(nfiles):
    traj = trajectoryTools.loadTrajectory(fnamebase, i)
    trajs.append(traj)    
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

In [ ]:
# Setup for discrete trajectory (in this case patchy dimer)
numParticles = 2
bufferSize = effectivetimeSteps
discretizator = msmrd2.trajectories.patchyDimer(numParticles, effectivetimeSteps)
# Set boundary (important for discretizer)
boxBoundary = msmrd2.box(boxsize, boxsize, boxsize, boundaryType)
discretizator.setBoundary(boxBoundary)

In [ ]:
# Generate discrete trajectories using msmrd2.trajectories.discrete
dtrajs = []
for i in range(nfiles):
    currentTraj = trajs[i]
    dtraj = discretizator.discretizeTrajectory(currentTraj)
    dtrajs.append(dtraj)
    if i%20 == 0:
        print("File ", i+1, " of ", nfiles, " done.", end="\r")
print("Done")

In [ ]:
# Write discrete trajectory to xyz file
for i, dtraj in enumerate(dtrajs):
    datafile  = open(fnamebase + str(i).zfill(4) + '_discrete_python2.xyz', 'w')
    for j in range(len(dtraj)):
        datafile.write(str(dtraj[j]) + '\n')
    datafile.close()
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

In [ ]:
# Alternatively, export data to file with pickle (so it can be loaded later)
#pickle_out = open("pickled_data/discretetrajs_patchydimer_t" + "{:.2E}".format(totalTimeSteps ) + "_s" + "{:d}".format(stride) + ".pickle","wb")
#pickle.dump(dtrajs, pickle_out)
#pickle_out.close()